In [100]:
import urllib.request
import re
import sys
import codecs
import json
from datetime import datetime
import os
import glob
import utils
import time
import getopt
import ssl
import numpy as np
import pandas as pd

In [8]:
# 设置参数

g_token = "c68bb70c102d863b3d96bc5b8901ff7689dfcba68f040b4fb3d56e9b2f8dbab8"
g_app_key = "e70d4c831ca1c79442f7ef7a68aeb9e3"
g_user_id = "5c16ff1d6e249c62d057987f"
# g_board_id = "5ac3159ed146b0793ff30206" # ThunderDB
g_board_id = "5b640e00c8b518181d80fa46" # Rating system
g_crontab_style = False

workload_pattern = u'[(（]\s*(\d+(?:\.\d+)?)\s*(pt|PT|pT|Pt)\s*[)）]' # 正则表达式
requirement_pattern = u'[\[【［]\s*(.*)\s*[】］\]]\s*' 

In [9]:
def basic_replace(url): # 替换 url 等主要参数
    url = url.replace("_APP_KEY_", g_app_key)
    url = url.replace("_TOKEN_", g_token)
    return url

def do_request(url): 
    context = ssl._create_unverified_context()
    request = urllib.request.Request(url) # urllib 网页抓取的库
    content = None
    flag = False

    for i in range(3):  # retry when network access failed 等于说这里是尝试了三遍
        try:
            response = urllib.request.urlopen(request, context= context) # 会验证一次 SSL 证书, 当目标使用的是自签名的证书时就会报 SSL 错误
            content = response.read().decode()
            flag = True # flag 应该是用来判断 try 是否执行成功
            break
        except urllib.request.HTTPError as e: # 给出报错信息
            print("http error: " + str(e))
        except Exception as e:
            print("error: " + str(e))

    if not flag:
        sys.exit(-1) # 没有 flag 说明报错了，然后退出程序

    if content is None:
        return []
    else:
        return json.loads(content)
    
def get_card_name_by_pattern(card_name, pattern):
    return re.findall(pattern, card_name, re.S | re.U) # 搜索字符串，以列表类型返回全部能匹配的子串

def fetch_board_by_user():
    url = 'https://api.trello.com/1/members/_USERID_/boards?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_USERID_", g_user_id)
    body = do_request(url)
    board_info = {}

    for item in body:
        board_info[item['shortLink']] = item['name']

    return board_info # 拿到 board 的 name 和 shortlink 意味着可以放进 url 直接访问页面

def insertStr(str_name): 
    
    if "[" in str_name: # 沙雕了。。。“[]”是意味着两个连在一起的
        str_list = list(str_name) # 字符数，可以利用这个在某个位置插入字
        str_pos = str_list.index('[') # 找到[ 的位置
        str_list.insert(str_pos, '\\') # 插入要插入的字符
        str_2 = "".join(str_list) # 将 list 转为 str
    
    else:
        str_2 = str_name
    
    return str_2

In [10]:
def fetch_list_id_by_board(list_pattern): # list_pattern 作为参数要输入 list name 
    url = 'https://api.trello.com/1/boards/_BOARDID_/lists?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url) # 返回那个 board 里所有的 list 信息
    list_info = []
    list_name = insertStr(list_pattern) # 识别 "[]" 
    
    for item in body:
        
        if re.search(list_name, item["name"], re.I) is not None: # re.search 在一个字符串中搜索匹配正则表达式的第一个位置，返回match对象
            list_info.append({'id': item['id'], 'name': item['name']}) 
            

    return list_info # 返回 list name 和 list_id


In [11]:
def fetch_members_by_board(): # 拿到所有成员的名称和 id
    url = 'https://api.trello.com/1/boards/_BOARDID_/members?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_BOARDID_", g_board_id)
    body = do_request(url)
    all_members_info = {}

    for item in body:
        all_members_info[item['id']] = item['fullName']

    return all_members_info # 返回 id 和 member name

In [12]:
def fetch_cards_by_list_id(list_id): #  可以通过上面的 fetch_list_id_by_board 可以获取 list_id
    url = 'https://api.trello.com/1/lists/_LISTID_/cards?key=_APP_KEY_&token=_TOKEN_'
    url = basic_replace(url)
    url = url.replace("_LISTID_", list_id) 
    body = do_request(url)

    return body # 用 list_id 获取 cards 的所有信息

In [94]:
def get_cards_info(list_id, board_members):

    available_cards_info = []

    all_cards_info = fetch_cards_by_list_id(list_id)

    for item in all_cards_info:
        card_info = { # 设置很多空的 dict，然后依据后面的判断填入一些参数
            'card_name': item['name'],
            'card_id': item['id'],
            'member_name': None,
            'member_id': None,
            'label_name': 'None',
            'label_id': None,
            'plan_hours': 0,
            'actual_hours': 0, # 现在好像没有分 actual hours 和 plan 
            'description' : None
        }

        idMembers = item['idMembers'] # 让 cards_info get 到 member 的信息
        if len(idMembers) > 0 and idMembers[0] is not None:
            member_id = idMembers[0]
            card_info['member_id'] = member_id
            card_info['member_name'] = board_members[member_id] # 这里不是报错是和 fetch_members_by_board 一起获得

        labels = item['labels']

        if len(labels) > 0 and labels[0] is not None: # 处理卡片的 labels
            label = labels[0]
            card_info['label_id'] = label['id']
            card_info['label_name'] = list(value['name'] for value in item['labels'])

        card_hours = get_card_name_by_pattern(item['name'], workload_pattern)
        
        if len(card_hours) == 2:
            card_info['actual_hours'] = float(card_hours[1][0]) * 2
            card_info['plan_hours'] = float(card_hours[0][0]) * 2
        else:
            card_info['plan_hours'] = float(card_hours[0][0]) * 2
            card_info['actual_hours'] = float(card_hours[0][0]) * 2

        
        card_info['description'] = item['desc']
        available_cards_info.append(card_info)
                       
    return available_cards_info
                       

In [97]:

board_members = fetch_members_by_board()
one_week_list = get_cards_info('5c998941abecaf7779f84b72',board_members)

In [109]:
df = pd.DataFrame(one_week_list, columns=['card_name',                                 
                                          'label_name',                                       
                                          'member_name',
                                          'plan_hours',
                                          'actual_hours',
                                          'description',
                                          'card_id',
                                          'label_id',
                                          'member_id',
                                         ])

In [110]:
df

,card_name,label_name,member_name,plan_hours,actual_hours,description,card_id,label_id,member_id
0,微信公号评论以及 quick start 引导（2pt）（3pt）,"[设计, 前端]",Chenxi LI,4.0,6.0,,5cf0d456f9f8b42320b542ce,5b640e009c16fb124a91ed29,5c16ff1d6e249c62d057987f
1,CovenantSQL wiki 中英文页面（4pt）,[PR],Chenxi LI,8.0,8.0,中文审核已经通过，英文前面还有 3000 多个 pending admission 慢慢等吧...,5cecf633aca9b032a72f7aed,5d26b72803aefe3969f34f76,5c16ff1d6e249c62d057987f
2,阿里云市场调研（4pt）,[后台服务],Chenxi LI,8.0,8.0,,5ceb48155bffef8024cbcebc,5b640e009c16fb124a91ed2a,5c16ff1d6e249c62d057987f
3,DBDB analysis 文章深入调研并修改（4pt）（2pt）,None,Chenxi LI,8.0,4.0,,5ce20129babea709b32fe440,None,5c16ff1d6e249c62d057987f
4,Quandl & Tushare 代码 Review （2pt）,[前端],Chenxi LI,4.0,4.0,Quandl：https://gitlab.com/thunderdb/quandldonw...,5ce64f0c28c0736452bf7759,5b640e009c16fb124a91ed28,5c16ff1d6e249c62d057987f
5,讨论并改进金融数据文章（4pt）,None,Chenxi LI,8.0,8.0,,5ce3a64909804e469a97f2bc,None,5c16ff1d6e249c62d057987f
6,推广 CovenantSQL 金融数据的文章初稿（4pt）,None,Chenxi LI,8.0,8.0,为 CQL 上的金融数据做 promotion，突出 CQL 的特点和易用性。,5ce1ff7f86884848784b2ba5,None,5c16ff1d6e249c62d057987f
7,调研bitmex永续合约的实现方式(8pt),[数据计算],朱劭劼,16.0,16.0,主要调研永续合约在产品实现上的方式以及做市细节，输出文档,5c8e786d3a031a1da8bd0cda,5b640e009c16fb124a91ed27,5b31d9729d873ba558d37004


In [ ]:
本周计划：


- 看完 youtube 视频
- 搞定剩下的爬虫
- 学完正则
- 试试 Linux
- 还有 Java
- 银行卡和信用卡
